## Web2Quiz: Generator Quiz from webpage content.

In [ ]:
#.env upload
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install dotenv


In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not api_key.startswith("sk-proj-"):
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
!pip install openai

In [ ]:
!pip install requests beautifulsoup4
!pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import requests
from tempfile import mkdtemp
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

class Website:
    def __init__(self, url, use_selenium=False):
        """
        Create Website object from the given URL.
        If use_selenium=True, fetch page with Selenium.
        Otherwise, use requests + BeautifulSoup.
        """
        self.url = url
        self.title = ""
        self.text = ""
        self.use_selenium = use_selenium

        if self.use_selenium:
            html = self._fetch_with_selenium()
        else:
            html = self._fetch_with_requests()

        if not html:
            self.title = "Error fetching page"
            self.text = "Could not retrieve HTML content."
            return

        soup = BeautifulSoup(html, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"

        # content_div = soup.find('div', id='content')
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input", "header", "footer", "nav", "aside"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = "No body tag found in the HTML."

    # Basic html scrapper
    def _fetch_with_requests(self):
        """Fetch HTML using requests."""
        try:
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(self.url, headers=headers, timeout=10)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching with requests: {e}")
            return None

    # Dynamic html scrapper
    def _fetch_with_selenium(self):
        """Fetch HTML using Selenium with improved options."""
        options = Options()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--headless")
        options.add_argument(f"--user-data-dir={mkdtemp()}")

        driver = None
        try:
            driver = webdriver.Chrome(options=options)
            driver.get(self.url)

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            html = driver.page_source
            return html
        except Exception as e:
            print(f"An error occurred during Selenium fetch: {e}")
            return None
        finally:
            if driver:
                driver.quit()



In [ ]:
site1 = Website("https://en.wikipedia.org/wiki/Integration_testing", use_selenium=False)
print("Title:", site1.title)
print("Text preview:", site1.text[:200])

site2 = Website("https://www.tpointtech.com/java-for-loop", use_selenium=True)
print("Title:", site2.title)
print("Text preview:", site2.text[:200])

In [ ]:
# Step 1: Create your prompts
system_prompt = f"You are a MCQ quiz generator. Analyze the provided TEXT and filter CONTENT relevent to {site1.title}. Then based on the relevant CONTENT generate 10 MCQs. List all correct options at the end."
user_prompt = f"Below is provided TEXT : \n{site1.text}"

# Step 2: Make the messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

# Step 4: print the result
response = openai.chat.completions.create(model="qwen/qwen2.5-vl-72b-instruct:free", messages=messages)
print(response.choices[0].message.content)

In [ ]:
# Step 1: Create your prompts
system_prompt = f"You are a MCQ quiz generator. Analyze the provided TEXT and filter CONTENT relevent to {site2.title}. Then based on the relevant CONTENT generate 10 MCQs. List all correct options at the end."
user_prompt = f"Below is provided TEXT : \n{site2.text}"

# Step 2: Make the messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

# Step 4: print the result
response = openai.chat.completions.create(model="qwen/qwen2.5-vl-72b-instruct:free", messages=messages)
print(response.choices[0].message.content)